# load file 

In [1]:
import json
def load_data(file_path):
        '''load json file and return data'''
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data

# preprocessing pipeline

In [1]:
from camel_pipeline import preprocess_text
text="تُمكّنك هذه الخدمة من تفعيل بطاقتك التموينية. الأسرة"
pre=preprocess_text(text)
pre = " ".join(pre)

pre

'تمكنك الخدمه تفعيل بطاقتك التموينيه الاسره'

# extract keywords from string
- list of doument 
-

In [2]:
from camel_pipeline import extract_keywords_tfidf
extract_keywords_tfidf([pre])

[['تمكنك', 'تفعيل']]

# extract words 
- بنجمع كله مع بعض وناخد الحذر لو فيه اكتر من terms علشان كدا بنعملها join 
-بعدين بنضيف full text ل كل خدمه 
-وبعدين بنضيف short text عباره عن نفس اللي فات بس من غير terms 
-وبعدين بنتضرب علي full text علشان نكتر documents كتير علشان tfidf يبقي accurate اكتر 
-وبعدين يعمل transform علي shortwords ويطلع اكتر كلمتين مهمين 

In [2]:
import json
import pickle
from camel_pipeline import enrich_services_with_texts, extract_keywords_from_short_texts_with_vectorizer
from scipy import sparse

# Step 1: Load services
with open("../scrape/scraped_services_data.json", "r", encoding="utf-8") as f:
    text = json.load(f)

# Step 2: Enrich with full_text & short_text
enriched_services = enrich_services_with_texts(text)

# Step 3: Extract keywords and get trained vectorizer
services_with_keywords, vectorizer = extract_keywords_from_short_texts_with_vectorizer(enriched_services)

# Step 3.5: Create and save TF-IDF matrix for full_text (for fast similarity search)
full_texts = [
    " ".join(service["full_text"]) if isinstance(service["full_text"], list) else service["full_text"]
    for service in services_with_keywords
]
tfidf_matrix = vectorizer.transform(full_texts)

# Save the TF-IDF matrix as sparse npz file
sparse.save_npz("service_tfidf_matrix.npz", tfidf_matrix)

# Step 4: Save keywords + services
with open("services_with_keywords.json", "w", encoding="utf-8") as f:
    json.dump(services_with_keywords, f, ensure_ascii=False, indent=4)

# Step 5: Save the trained TF-IDF model
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
